In [3]:
import pandas as pd

from fleet.utils import data
from fleet.base_schemas import TorchModelSpec

model_path = 'tests/data/yaml/small_regressor_schema.yaml'
csv_path = 'tests/data/csv/zinc_extra.csv'

model_spec = TorchModelSpec.from_yaml(model_path)
df = pd.read_csv(csv_path)


dataset = data.MarinerTorchDataset(
    data=df,
    dataset_config=model_spec.dataset,
    model_config=model_spec.spec
)


print(dataset.get_subset_idx())

[FleetmoleculefeaturizerLayerConfig(type='fleet.model_builder.featurizers.MoleculeFeaturizer', name='MolToGraphFeaturizer', constructor_args=FleetmoleculefeaturizerConstructorArgs(allow_unknown=False, sym_bond_list=True, per_atom_fragmentation=False), forward_args=FleetmoleculefeaturizerForwardArgsReferences(mol='$smiles'))] []
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,

/home/vilma/github.com/trident-bio/mariner/backend/fleet/utils/data.py:153: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(arr)


In [3]:
from fleet.model_functions import fit
import pandas as pd

fit(
    spec=spec,
    train_config=train_config,  # todo: make this type narrower
    mlflow_model_name="small regressora",
    mlflow_experiment_name="small regressor training",
    dataset=pd.read_csv(csv_path)
)

ERROR    [model_functions.py:84] RestException('RESOURCE_ALREADY_EXISTS: Experiment(name=small regressor training) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)\n(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "experiments_name_key"\nDETAIL:  Key (name)=(small regressor training) already exists.\n\n[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (%(name)s, %(artifact_location)s, %(lifecycle_stage)s, %(creation_time)s, %(last_update_time)s) RETURNING experiments.experiment_id]\n[parameters: {\'name\': \'small regressor training\', \'artifact_location\': \'\', \'lifecycle_stage\': \'active\', \'creation_time\': 1684263213744, \'last_update_time\': 1684263213744}]\n(Background on this error at: https://sqlalche.me/e/14/gkpj)')


RuntimeError: Failed to create mlflow experiment